In [1]:
result = os.system('pip install gradio')
print(result)

NameError: name 'os' is not defined

In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
import random

In [4]:
# Sample labels and messages 
labels = ['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']
awareness_messages = {
    'toxicity': [
        "Toxic comments can deeply hurt others—choose empathy.",
        "What you say online matters. Avoid spreading negativity.",
        "Toxicity can damage communities. Lead with kindness.",
    ],
    'severe_toxicity': [
        "This level of toxicity can be extremely harmful. Please reconsider your words.",
        "Such extreme language might get flagged or reported—stay respectful.",
        "Severe toxicity can lead to serious consequences, even legal action.",
    ],
    'obscene': [
        "Obscene language can make others uncomfortable. Let's keep it respectful.",
        "Using vulgar terms doesn't strengthen your point.",
        "Consider expressing yourself in a cleaner way.",
    ],
    'threat': [
        "Threats aren't just harmful—they can be illegal.",
        "Making others feel unsafe is never acceptable.",
        "This might be interpreted as a threat. Please stay kind.",
    ],
    'insult': [
        "Insulting others reflects poorly on you, not them.",
        "Words can wound. Please avoid using personal attacks.",
        "Disagreements are okay—insults are not.",
    ],
    'identity_attack': [
        "Targeting someone's identity promotes hate. Please don't.",
        "Disrespecting who someone is can leave lifelong scars.",
        "We are all different. Let's celebrate that—not attack it.",
    ],
    'sexual_explicit': [
        "Explicit content might not be appropriate for all audiences.",
        "Sexual language can cross boundaries—respect others' comfort zones.",
        "Please consider whether this message is suitable for public discussion.",
    ],
}

In [12]:
label_to_image = {
    'toxicity': "images/toxicity.PNG",
    'severe_toxicity': "images/severe_toxicity.PNG",
    'obscene': "images/obscene.PNG",
    'threat': "images/threat.PNG",
    'insult': "images/insult.PNG",
    'identity_attack': "images/identity_attack.PNG",
    'sexual_explicit': "images/sexual_explicit.PNG"
}

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


In [16]:
label_to_link_image = {
    'toxicity': "https://drive.google.com/uc?export=view&id=1VB-3ieR2i1-Kk3oJrfQI1AVSoBY9fV8g",
    'severe_toxicity': "https://drive.google.com/uc?export=view&id=1LzXCKkyJaFK6H87cwGcn567lhVQ9I6bh",
    'obscene': "https://drive.google.com/uc?export=view&id=1lO6Uwd1o4k1ujLKDcpUzW1wsxmxVi0Y0",
    'threat': "https://drive.google.com/uc?export=view&id=1lk5EgFCvmoZi310Ih8_wKuGjhHzh_0Z1",
    'insult': "https://drive.google.com/uc?export=view&id=1FtbDj3Eoy3F97uSdXhgjVsD61zc0H3T2",
    'identity_attack': "https://drive.google.com/uc?export=view&id=1lENZaCcXCK_VIh689SMtHyn7msZYlfTR",
    'sexual_explicit': "https://drive.google.com/uc?export=view&id=19bj0Vd-qF18TZ9fwhI7SCVBxB3rNXREe"
}

In [22]:
label_to_link_image2 = {
    'toxicity': "https://i.imgur.com/LYWCuH3.png",
    'severe_toxicity': "https://i.imgur.com/YT8roF8.png",
    'obscene': "https://i.imgur.com/Rkb3sYh.png",
    'threat': "https://i.imgur.com/YmEEHJ1.png",
    'insult': "https://i.imgur.com/A4iq2Km.png",
    'identity_attack': "https://i.imgur.com/SjIAa6F.png",
    'sexual_explicit': "https://i.imgur.com/pATebIx.png"
}

In [17]:
# load model
model = tf.keras.models.load_model("last_model.keras", compile=False)


In [18]:
import numpy as np
import pickle
from tensorflow.keras.layers import TextVectorization

In [19]:
# load vocabulary
with open('vectorizer_vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)

# Vectorizer setup from vocabulary
MAX_WORDS = 40000  
SEQUENCE_LENGTH = 600  

vectorizer = TextVectorization(
    max_tokens=MAX_WORDS,
    output_sequence_length=SEQUENCE_LENGTH,
    output_mode='int'
)

vectorizer.set_vocabulary(vocabulary)

In [65]:
def classify_and_advise(text):
    X = vectorizer([text])
    prediction = model.predict(X)[0]

    detected_cards = []
    for i, prob in enumerate(prediction):
        if prob >= 0.5:
            label = labels[i]
            img_url = label_to_link_image2.get(label)
            msg = random.choice(awareness_messages.get(label, ["Be respectful online."]))
            detected_cards.append(f"""
            <div class='card'>
                <img src="{img_url}" alt="{label}">
                <h3 class='category-title'>{label.replace('_', ' ').title()}</h3>
                <p class='category-msg'>{msg}</p>
            </div>
            """)

    if not detected_cards:
        return """
        <div style='display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 40px; border-radius: 12px;'>
            <img src="https://clipart-library.com/images/8cA6Ga9Ki.jpg" alt="Good" width='200' height='200' style='border-radius: 20px; box-shadow: 0 0 10px gray;'>
            <h2 style='font-size: 30px;'>This comment looks safe and respectful!</h2>
            <p style='font-size: 18px;'>Thanks for spreading positivity online </p>
        </div>
        """

   
    return f"""
    <style>
        .card-container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            gap: 20px;
            margin-top: 20px;
        }}
        .card {{
            width: 260px;
            padding: 15px;
            border-radius: 12px;
            text-align: center;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }}
        .card img {{
            width: 200px;
            height: 200px;  
            object-fit: cover; 
            border-radius: 12px;
            margin-bottom: 10px;
        }}
        .category-title {{
            color: #cc0000 !important;
            font-weight: bold;
            font-size: 30px !important;
            margin-bottom: 10px;
        }}
        .category-msg {{
            font-size: 18px;  
            font-family: Arial, sans-serif;
        }}
    </style>
    <div class='card-container'>
        {''.join(detected_cards)}
    </div>
    """

In [66]:
# Define the Gradio interface
with gr.Blocks() as demo:

    gr.Markdown("<div style='text-align:center; color:#cc0000'><h1>Toxicity Detection Game</h1></div>")
    gr.Markdown("<div style='text-align:center;'><h3>Type a comment and see how toxic it is — and how it emotionally impacts others.</h3></div>")

    output_html = gr.HTML()
    user_input = gr.Textbox(lines=4, placeholder="Write your comment here...", label="Your Comment")
    analyze_button = gr.Button("Analyze Comment")

    analyze_button.click(classify_and_advise, inputs=user_input, outputs=output_html)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7887
Running on public URL: https://edf404f07a063c9cf5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
